# documentation 

generate and provide backend for services demonstrated in the documention for these notebooks



## convert notebooks to API


convert notebook to open api?


TODO: add check for readme.md in notebooks for description pages

TODO: finish export and convert other documentation to the open API format



#### the code

ROUTE[] = /swagger

ROUTE[] = /swagger/v1/swagger.json

TODO: console.log really good use of what attributes should be used for



In [ ]:
const path = require('path')
const package = require('../package.json')
//var getRpcFromSpec = importer.import('get rpc from spec')
//var getPermissions = importer.import('rpc permissions')
//var groupPermissions = importer.import('test rpc permissions')
//var getParameters = importer.import('get parameters')
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { cacheCells } = importer.import('cache notebook')
const parseEnvironment = importer.import('parse environment')

// TODO: move this to patterns utility
function getTail(path, ctx) {
    let cur = path.split('.')[0]
    if (!cur) return ctx
    if (typeof ctx[cur] === 'undefined') ctx[cur] = {}
    return getTail(path.split('.').slice(1).join('.'), ctx[cur])
}

function getPathId(cell) {
    return path
        .relative('../', cell.filename)
        .replace(/\.ipynb|\s+/ig, '')
        .replace(/[^a-z0-9_\/]/ig, '')
        .replace(/\//ig, '.resources.')
        .toLowerCase()
}

function getRpcSpecFromCells(search) {
    //var functions = groupPermissions(['Function', 'Available'], search)

    let then = Date.now()
    let cells
    if (search) {
        console.log('searching:', search)
        cells = importer.interpret(search)
        console.info('searching took:', ((Date.now() - then) / 1000) + 's')
    } else {
        console.log('searching for every notebook:', then)
        const cellCache = importer.import('cell cache').cellCache
        cells = cellCache.map(cache => importer.lookupCell(cache[1], cacheCells))
        console.info('searching every notebook took:', ((Date.now() - then) / 1000) + 's')

    }

    then = Date.now()
    console.log('structuring every resource:', then)
    let resources = cells.reduce((obj, cell) => {
        let cache = functionCache[path.join(path.dirname(cell.filename), cell.id)]
        let env = parseEnvironment(cell.markdown)
        if(((!cache || !cache.exports || !cache.exports[0]) && !env['ROUTE']) || !cell.questions || !cell.questions[0]) {
            if(cell.questions && cell.questions[0] == 'linkedin home stylesheet') {
                console.log(!cache , !env['ROUTE'] , !cell.questions)
            }
            return obj
        }
        // TODO: multi lingual parameters?
        // TODO: add markdown ROUTE getEnvironment here
        let route = typeof env['ROUTE'] == 'string' ? env['ROUTE'] : (env['ROUTE'] || [])[0]
        if(route) {
            route = route.replaceAll(/^\/|\/$/g, '').split('/')[0]
        }
        let fn = ((cache || {}).exports || [])[0] || route
        let pathId = getPathId(cell)
        let tail = getTail(`${pathId}`, obj)
        if (typeof tail.methods === 'undefined') tail.methods = {}
        tail.methods[fn] = {
            description: (cache || {}).summary || cell.markdown,
            scopes: ['Function', 'Available', 'Public'],
            id: `${package.name}.${pathId}.${fn}`,
            httpMethod: 'GET',
            path: `?function=${cell.questions[0]}`,
            parameters: ((cache || {}).exports || []).slice(1).reduce((o, p) => (o[p] = {
                type: 'string',
                location: 'query',
                required: true,
                description: ''
            }, o), {})
        }
        return obj
    }, {})
    console.log('structuring every resource took:', ((Date.now() - then) / 1000) + 's')

    return {
        title: 'Jupyter Ops',
        name: package.name,
        description: package.description,
        protocol: 'rest',
        baseUrl: process.env.FUNCTIONS_URL,
        resources: resources,
        auth: {},
        schemas: {},
        parameters: {}
    }
}

module.exports = getRpcSpecFromCells;


#### test convert notebook api?


In [6]:
var importer = require('../Core');
var getRpcSpecFromCells = importer.import('convert notebook to open api');
var getRpcFromSpec = importer.import('get rpc from spec');
var getEnvironment = importer.import('get environment');
var groupPermissions = importer.import('test get permissions');
getEnvironment('STUDY_LOCAL')

if(typeof $$ !== 'undefined') {
    $$.async()
    var spec = getRpcSpecFromCells(['study sauce.ipynb', 'rpc.ipynb'])
//    var spec = getRpcSpecFromCells('study sauce.ipynb');
    console.log(spec)
    var juypter_ops = getRpcFromSpec(spec)
    console.log(juypter_ops)
//    juypter_ops.marketing.studysauce.copyStudy({email: 'megamindbrian@gmail.com'})
    juypter_ops.core.rpc.getPermissions({search: null})
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


importing convert notebook to open api - 1 cell - documentation.ipynb[0]
compiling /Users/briancullinan/jupyter_ops/Frontends/documentation.ipynb[0] aka convert notebook to open api with module,importer,exports,__dirname,__filename,$$
Error: Line 2: Unexpected token :
    at ErrorHandler.constructError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:5012:22)
    at ErrorHandler.createError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:5028:27)
    at Parser.unexpectedTokenError (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:1985:39)
    at Parser.throwUnexpectedToken (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:1995:21)
    at Parser.consumeSemicolon (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:2297:23)
    at Parser.parseExpressionStatement (/Users/briancullinan/jupyter_ops/node_modules/esprima/dist/esprima.js:3649:15)
    at Parser.parseStatement (/Users/briancu

Error: connect ECONNREFUSED 127.0.0.1:8010

### convert google discovery to documentation?

https://developers.google.com/apis-explorer/#s/discovery/v1/discovery.apis.getRest?api=discovery&version=v1&_h=2&

https://www.googleapis.com/discovery/v1/apis/discovery/v1/rest

https://developers.google.com/apis-explorer/#p/discovery/v1/discovery.apis.list?_h=1&




discover well-known apis?

TODO: figure out how OpenAPI, Swagger RAML do this, Google's is obvious

https://tools.ietf.org/html/rfc5785


## Export docs


### export documentation?


In [ ]:
const fs = require('fs')
const path = require('path')
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { listInProject } = importer.import('list project files');
const { cacheCells } = importer.import('cache notebook')
const { safeurl } = importer.import('domain cache tools')
const { Remarkable } = require('remarkable');
const md = new Remarkable({ html: true, xhtmlOut: true, breaks: true });

const PROJECT_PATH = path.resolve(path.join(path.dirname(__dirname), 'docs'))
const INTERPRET = {}

async function exportAll(query) {
  // await storeAllLlamaFunctions()
  if (!fs.existsSync(PROJECT_PATH)) {
    fs.mkdirSync(PROJECT_PATH)
  }

  // TODO: get a list of all notebooks
  let notebooks = listInProject(path.resolve(__dirname, '../'), '{,*,*/*,*/*/*,*/*/*/*}*.ipynb')

  let languageTemplate = importer.interpret('highlight js')
  let javadocTemplate = importer.interpret('javadoc template')

  // TODO: generate an index page that describes the index.js and __init__.py loaders

  let results = {}
  const search = path.join(PROJECT_PATH, 'search.html')
  results[search] = {
    output: '',
    indexOutput: '',
    title: 'Search',
    parent: '<a href="./index.html">Home</a>',
    next: '',
    prev: ''
  }
  results[search].index = search
  results[search].output += '<h1>Search</h1>\n\n'
  const intro = path.join(PROJECT_PATH, 'index.html')
  results[intro] = {
    output: '',
    indexOutput: '',
    title: 'Categories',
    parent: '',
    next: '',
    prev: ''
  }
  results[intro].output += '<h1>Categories</h1>\n\n'

  let prevCategory = ''
  let prevIndex = ''
  let namespaces = {}
  for (let i = 0; i < notebooks.length; i++) {
    // TODO: output folders and pages for every namespace and notebook
    const relative = path.relative(path.resolve(__dirname, '../'), notebooks[i])
    let category = relative.split(path.sep)[0]
    if (category.endsWith('.ipynb')) {
      category = 'Introduction'
    }

    if (!fs.existsSync(path.join(PROJECT_PATH, category))) {
      fs.mkdirSync(path.join(PROJECT_PATH, category))
    }

    const catpath = path.join(PROJECT_PATH, category, 'index.html')
    if (typeof results[catpath] == 'undefined') {
      prevIndex = '' // start fresh with nav
      results[intro].output += '<a href="./' + category + '/index.html">' + category + '</a>' + '\n<br /><br />\n'
      results[intro].indexOutput += '<a href="../' + category + '/index.html">' + category + '</a>' + '\n<br /><br />\n'
      results[search].indexOutput += '<h3><a href="./' + category + '/index.html">' + category + '</a></h3>'
      results[catpath] = {
        output: '',
        indexOutput: '',
        title: category,
        next: '',
        prev: ''
      }
      results[catpath].index = intro
      results[catpath].parent = '<a href="../index.html">Categories</a>'
      results[catpath].output += '<h1>' + category + '</h1>\n\n'
      results[search].output += '<h2>' + category + '</h2>\n\n'
      results[catpath].link = '<a href="../' + category + '/index.html">' + category + '</a>'
      if (i > 0 && prevCategory) {
        results[prevCategory].next = results[catpath].link
        results[catpath].prev = results[prevCategory].link
      }
      prevCategory = catpath
    }


    const namespace = path.basename(notebooks[i]).replace('.ipynb', '')
    const fullpath = path.join(PROJECT_PATH, category, safeurl(namespace))
    if (!fs.existsSync(fullpath)) {
      fs.mkdirSync(fullpath)
    }

    results[catpath].output += '<a href="./' + safeurl(namespace) + '/index.html">' + namespace + '</a>' + '\n<br /><br />\n'
    // used by the next level down
    results[catpath].indexOutput += '<a href="../' + safeurl(namespace) + '/index.html">' + namespace + '</a>' + '\n<br /><br />\n'
    const index = path.join(fullpath, 'index.html')
    results[index] = {
      output: '',
      indexOutput: '',
      title: namespace,
      next: '',
      prev: ''
    }
    results[index].index = catpath
    results[index].parent = '<a href="../../' + category + '/index.html">' + category + '</a>'
    results[index].output += '<h1>' + namespace + '</h1>\n\n'
    results[search].output += '<h3>' + namespace + '</h3>\n\n'
    results[search].indexOutput += '<h4><a href="./' + category + '/' + safeurl(namespace) + '/index.html">' + namespace + '</a></h4>'
    results[index].link = '<a href="../' + safeurl(namespace) + '/index.html">' + namespace + '</a>'
    if (i > 0 && prevIndex) {
      results[prevIndex].next = results[index].link
      results[index].prev = results[prevIndex].link
    }
    prevIndex = index

    namespaces[relative] = []

    const newCache = cacheCells(notebooks[i])


    let prevKey = ''
    for (let j = 0; j < newCache.length; j++) {
      let cellname = newCache[j].questions[0]
      if (!cellname) {
        cellname = 'cell_' + j
      } else {
        cellname = safeurl(cellname)
      }
      const key = path.join(fullpath, cellname + '.html')
      results[key] = {
        output: '\n\n<pre class="' + newCache[j].language + '"><code>' + newCache[j].code.replaceAll('&', '&amp;').replaceAll('<', '&lt;').replaceAll('>', '&gt;').replaceAll(/(import_notebook|importNotebook|import)\([\\\n\r\[\]\s'"]*([\s\S]*?)[\\\n\r\]\[\s'"]*\)/gi, (match, ...args) => {
          let searches = args[1].split(/[\\\n\r\s'"]*,[\\\n\r\s'"]*/gi)
          let links = []
          for(let k = 0; k < searches.length; k++) {
            try {
              if(!searches[k].trim()) continue
              let cell
              if(typeof INTERPRET[searches[k]] != 'undefined') {
                cell = INTERPRET[searches[k]]
              } else {
                console.log('Searching: ', searches[k])
                cell = importer.interpret(searches[k])
                INTERPRET[searches[k]] = cell
              }
              if(!cell) continue
              const relative = path.relative(path.resolve(__dirname, '../'), cell.filename)
              links.push('"<a href="../../' + relative.split(path.sep)[0] 
                + '/' + safeurl(path.basename(cell.filename).replace('.ipynb', '')) 
                + '/' + safeurl(cell.questions[0]) + '.html">' + searches[k] + '</a>"')
            } catch(e) {}
          }
          return args[0] + '(' + links.join(',\n') + ')'
        }) + '</code></pre>\n\n'
      }
      results[key].output += languageTemplate.code.replaceAll('${LANGUAGE}', newCache[j].language)

      let title = newCache[j].questions[0]
      if (!title) {
        title = 'Cell ' + j
      }
      results[key].output = '<h1>' + title + '</h1>\n\n' + results[key].output
      if (newCache[j].questions[0])
        results[key].output = '<h2>Run example</h2>\n\n<pre language="bash"><code>npm run import -- "' + newCache[j].questions[0] + '"</code></pre>' + results[key].output
      
      results[key].index = index
      results[key].title = title
      results[key].link = '<a href="./' + cellname + '.html">' + title + '</a>'
      if (j > 0) {
        results[prevKey].next = results[key].link
        results[key].prev = results[prevKey].link
      } else {
        results[key].prev = ''
      }
      if (j == newCache.length - 1) {
        results[key].next = ''
      }

      results[key].parent = '<a href="../' + safeurl(namespace) + '/index.html">' + namespace + '</a>'
      results[search].output += '<a href="./' + category + '/' + safeurl(namespace) + '/' + cellname + '.html">' + title + '</a>' + '\n<br /><br />\n'
      results[search].indexOutput += '<a href="./' + category + '/' + safeurl(namespace) + '/' + cellname + '.html">' + title + '</a>' + '\n<br /><br />\n'
      results[index].output += '<a href="./' + cellname + '.html">' + title + '</a>' + '\n<br /><br />\n'
      results[index].indexOutput += '<a href="./' + cellname + '.html">' + title + '</a>' + '\n<br /><br />\n'
      prevKey = key

      const description = functionCache[notebooks[i] + '[' + j + ']']
      if (!description) {
        continue
      }

      results[search].output += md.render(description.summary.replaceAll(/^.*?summary.*?:/gmi, ''))
      results[index].output += md.render(description.summary.replaceAll(/^.*?summary.*?:/gmi, ''))
      if(description.amazing) {
        let codeMatch = description.amazing.replace(/^```(markdown)\n*|\n```$/gi, '')
        codeMatch = (/```(javascript|python|bash|[a-z0-9]+)\n*([\s\S]*?)\n```/gi).exec(codeMatch)
        if(codeMatch) {
          results[key].output += '<div class="gold"><h2>What the code could have been:</h2>\n<pre class="' + codeMatch[1] + '"><code>' + codeMatch[2] + '</code></pre></div>'
        } else {
          results[key].output += '<div class="gold"><h2>What the code could have been:</h2>\n<pre class="' + newCache[j].language + '"><code>' + description.amazing.replaceAll('&', '&amp;').replaceAll('<', '&lt;').replaceAll('>', '&gt;') + '</code></pre></div>'
        }
      }
      results[key].output = md.render(description.summary.replaceAll(/^.*?summary.*?:/gmi, '')) + results[key].output
      results[key].output += md.render(description.description)
    }

    // TODO: generate parameter, return type descriptions


  }


  // TODO: export all the markdown

  const paths = Object.keys(results)
  for (let i = 0; i < paths.length; i++) {
    // TODO: add navigation and full template
    let output = javadocTemplate.code
    output = output.replaceAll('${TITLE}', results[paths[i]].title)
    output = output.replaceAll('${PARENT}', results[paths[i]].parent)
    output = output.replaceAll('${NEXT}', results[paths[i]].next)
    output = output.replaceAll('${PREV}', results[paths[i]].prev)
    const relative = path.relative(path.resolve(__dirname, '../docs'), paths[i])
    const searchRelative = path.relative(path.dirname(relative), 'search.html')
    const pluginRelative = path.relative(path.dirname(relative), 'mergehtml.js')
    output = output.replaceAll('${SEARCH}', '<a href="' + searchRelative + '">Search</a>')
    if (results[paths[i]].index)
      output = output.replaceAll('${CLASSES}', results[results[paths[i]].index].indexOutput)
    else
      output = output.replaceAll('${CLASSES}', '')
    // do this last because it should already the necessary replacements
    output = output.replaceAll('${OUTPUT}', results[paths[i]].output)
    output = output.replaceAll('${PLUGIN}', pluginRelative)
    fs.writeFileSync(paths[i], output)
  }

}

module.exports = exportAll



highlight js?

so someday these can be like mustache templates or w/e


In [ ]:
<link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/styles/atom-one-dark.min.css">


<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/highlight.min.js"></script>

<!-- and it's easy to individually load additional languages -->
<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/languages/${LANGUAGE}.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.9.0/languages/bash.min.js"></script>
<script src="${PLUGIN}"></script>

<script>
let codeCells = document.querySelectorAll('code')
let startLine = 0
for(let i = 0; i < codeCells.length; i++) {
  let lineCode = document.createElement('code')
  lineCode.classList.add('lines')
  let lines = codeCells[i].innerHTML.replace(/\n&nbsp;$/, '').split('\n')
  lineCode.innerHTML = lines.map((line, i) => '<p id="' + (startLine + i + 1) + '"><a class="number" href="#' + (startLine + i + 1) + '">' + (i + 1) + '</a>' + (line.trim().length == 0 ? '&nbsp;' : line) + '</p>').join('')
  codeCells[i].parentElement.insertAdjacentElement('afterend', lineCode)
  codeCells[i].addEventListener('dblclick', (function (html) {
    navigator.clipboard.writeText(html)
  }).bind(null, codeCells[i].innerText))
  startLine += lines.length
}
hljs.addPlugin(mergeHTMLPlugin);
hljs.highlightAll();

</script>




javadoc template?


In [ ]:
<!DOCTYPE html>
<html>

<head>
  <meta charset='utf-8'>
  <meta http-equiv='X-UA-Compatible' content='IE=edge'>
  <title>${TITLE}</title>
  <meta name='viewport' content='width=device-width, initial-scale=1'>
  <link rel='stylesheet' type='text/css' media='screen' href='main.css'>
  <style>
    html {
      padding: 0;
      margin: 0;
    }

    nav {
      position: fixed;
      overflow: auto;
      top: 0;
      left: 0;
      right: auto;
      bottom: 0;
      width: 200px;
    }

    header {
      background-color: #EEE;
      padding: 10px;
    }

    body {
      padding: 0 0 0 200px;
      margin: 0;
    }

    .gold pre code,
    .gold pre code span,
    .gold code pre,
    .gold code pre span {
      color: gold;
    }

    @media screen and (max-width: 600px) {
      body {
        padding-left: 0;
      }

      nav {
        display: none;
      }
    }
  </style>
</head>

<body>
  <nav>
    <h3>${PARENT}</h3>
    ${CLASSES}
  </nav>
  <header>
    ${PARENT} | ${PREV} | ${NEXT} | ${SEARCH}
  </header>

  ${OUTPUT}
</body>

</html>